In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable

import numpy as np

torch.manual_seed(1)

In [2]:
# Device configuration
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
# define corpus
corpus = [
    'he is a king',
    'she is a queen',
    'he is a man',
    'she is a woman',
    'warsaw is poland capital',
    'berlin is germany capital',
    'paris is france capital',
]

In [4]:
# step1. creating vocabulary

# tokenize sentences
def tokenize_corpus(corpus):
    tokens = [sentence.split() for sentence in corpus]
    return tokens

tokenized_corpus = tokenize_corpus(corpus)

print("tokenized corpus:", tokenized_corpus)

# word->idx, idx->word
word2idx = {}
for sentence in tokenized_corpus:
    for token in sentence:
        if token not in word2idx:
            word2idx[token] = len(word2idx)

idx2word = {v: k for k, v in word2idx.items()}

print("word2idx:", word2idx)
print("idx2word:", idx2word)

word_vocab_size = len(word2idx)

tokenized corpus: [['he', 'is', 'a', 'king'], ['she', 'is', 'a', 'queen'], ['he', 'is', 'a', 'man'], ['she', 'is', 'a', 'woman'], ['warsaw', 'is', 'poland', 'capital'], ['berlin', 'is', 'germany', 'capital'], ['paris', 'is', 'france', 'capital']]
word2idx: {'he': 0, 'is': 1, 'a': 2, 'king': 3, 'she': 4, 'queen': 5, 'man': 6, 'woman': 7, 'warsaw': 8, 'poland': 9, 'capital': 10, 'berlin': 11, 'germany': 12, 'paris': 13, 'france': 14}
idx2word: {0: 'he', 1: 'is', 2: 'a', 3: 'king', 4: 'she', 5: 'queen', 6: 'man', 7: 'woman', 8: 'warsaw', 9: 'poland', 10: 'capital', 11: 'berlin', 12: 'germany', 13: 'paris', 14: 'france'}


In [5]:
# step 2. Generate pairs (center word - context word)
window_size = 2
idx_pairs = []

# for each sentence
for sentence in tokenized_corpus:
    word_idxs = [word2idx[word] for word in sentence]
    for center_word_pos in range(len(word_idxs)):
        for context_word_pos in range(len(word_idxs)):
            if center_word_pos == context_word_pos:
                pass
            else:
                idx_pairs.append((word_idxs[center_word_pos], word_idxs[context_word_pos]))
idx_pairs = np.array(idx_pairs)
print(idx_pairs.shape)

(84, 2)


## Defining goal
@ We want to predict context, given center word and soma parametrization.

@ We are making P(context | center) close to 0 for all non-existing (context, center) pairs.

In [10]:
# input layer: just the center word encoded in one-hot manner.
def get_one_hot(word_idx, word_vocab_size):
    x = torch.zeros(word_vocab_size).float()
    x[word_idx] = 1.0
    return x

In [21]:
embedding_dim = 5

W1 = torch.randn((embedding_dim, word_vocab_size), requires_grad=True).to(device)
W2 = torch.randn((word_vocab_size, embedding_dim), requires_grad=True).to(device)

num_epochs = 500
lr = 0.1

print(W1)
print(W2)

tensor([[-0.1276, -0.1882,  1.6163, -1.0897, -0.0009,  0.5652, -0.0148,
          0.1821, -0.4875, -0.4971,  1.4690,  0.0375,  0.0658, -0.9131,
         -0.9776],
        [-1.1762,  1.3536,  1.6513, -0.0177, -1.2269,  0.3899,  1.0716,
          0.8904, -1.8149,  2.0859,  0.1051,  0.0199, -0.6802, -0.4352,
         -0.7243],
        [-0.7842, -2.7757,  0.9390,  0.8462,  0.2809, -1.3055,  0.5534,
          0.1960, -1.0108,  0.6361,  0.0423, -0.5353,  0.6595,  0.3424,
         -0.1554],
        [-1.4800,  1.5622, -2.1810, -0.1624,  1.7706,  2.3947, -1.0720,
          0.6024,  0.1352, -0.6365,  0.0678,  0.0344,  1.1384,  1.6472,
          0.0985],
        [-0.1218,  0.3274, -1.0497,  0.2502,  1.7113,  0.9126, -0.2896,
          1.4172, -0.4237, -0.0769,  0.5362,  0.6485,  0.1329, -0.1203,
         -1.1499]])
tensor([[ 0.5847, -1.3204, -0.2781, -1.4720,  1.2683],
        [ 0.7452, -0.3816, -1.8310, -1.4525,  0.1274],
        [ 1.6439, -0.3111, -1.0572, -1.6718,  1.2436],
        [ 1.4090, -

In [22]:
for epoch in range(num_epochs):
    loss_val = 0
    for center, context in idx_pairs:    
        input = get_one_hot(center, word_vocab_size).to(device).float()
        target = torch.from_numpy(np.array([context])).to(device).long()
        w1_out = torch.matmul(W1, input)
        w2_out = torch.matmul(W2, w1_out)

        log_softmax = F.log_softmax(w2_out, dim=0).view(1, -1)
        
        score, idx = torch.max(log_softmax, dim=1)
    
        print("predicted:", idx.item(), " truth:", target.item())
        loss = F.nll_loss(log_softmax, target)
        loss_val += loss.item()
        loss.backward()
        
        W1.data -= lr * W1.grad.data
        W2.data -= lr * W2.grad.data
        
        W1.grad.data.zero_()
        W2.grad.data.zero_()
        print("\n")
    
    if epoch % 10 == 0:
        print("[%d/%d] loss:%.3f" % (epoch+1, num_epochs, loss_val / float(len(idx_pairs))))
        

predicted: 14  truth: 1


predicted: 1  truth: 2


predicted: 1  truth: 3


predicted: 11  truth: 0


predicted: 11  truth: 2


predicted: 11  truth: 3


predicted: 14  truth: 0


predicted: 14  truth: 1


predicted: 14  truth: 3


predicted: 8  truth: 0


predicted: 8  truth: 1


predicted: 14  truth: 2


predicted: 3  truth: 1


predicted: 3  truth: 2


predicted: 3  truth: 5


predicted: 11  truth: 4


predicted: 4  truth: 2


predicted: 11  truth: 5


predicted: 14  truth: 4


predicted: 9  truth: 1


predicted: 14  truth: 5


predicted: 3  truth: 4


predicted: 3  truth: 1


predicted: 11  truth: 2


predicted: 0  truth: 1


predicted: 1  truth: 2


predicted: 2  truth: 6


predicted: 2  truth: 0


predicted: 2  truth: 2


predicted: 2  truth: 6


predicted: 5  truth: 0


predicted: 5  truth: 1


predicted: 5  truth: 6


predicted: 14  truth: 0


predicted: 14  truth: 1


predicted: 14  truth: 2


predicted: 3  truth: 1


predicted: 2  truth: 2


predicted: 2  truth: 7


predicted

predicted: 1  truth: 2


predicted: 2  truth: 1


predicted: 2  truth: 2


predicted: 2  truth: 5


predicted: 10  truth: 4


predicted: 10  truth: 2


predicted: 2  truth: 5


predicted: 1  truth: 4


predicted: 1  truth: 1


predicted: 1  truth: 5


predicted: 2  truth: 4


predicted: 4  truth: 1


predicted: 2  truth: 2


predicted: 1  truth: 1


predicted: 1  truth: 2


predicted: 1  truth: 6


predicted: 2  truth: 0


predicted: 2  truth: 2


predicted: 2  truth: 6


predicted: 1  truth: 0


predicted: 1  truth: 1


predicted: 1  truth: 6


predicted: 1  truth: 0


predicted: 1  truth: 1


predicted: 1  truth: 2


predicted: 2  truth: 1


predicted: 2  truth: 2


predicted: 2  truth: 7


predicted: 2  truth: 4


predicted: 2  truth: 2


predicted: 2  truth: 7


predicted: 1  truth: 4


predicted: 1  truth: 1


predicted: 1  truth: 7


predicted: 2  truth: 4


predicted: 2  truth: 1


predicted: 1  truth: 2


predicted: 9  truth: 1


predicted: 9  truth: 9


predicted: 9  truth: 10

predicted: 10  truth: 4


predicted: 10  truth: 2


predicted: 2  truth: 5


predicted: 1  truth: 4


predicted: 1  truth: 1


predicted: 1  truth: 5


predicted: 2  truth: 4


predicted: 4  truth: 1


predicted: 1  truth: 2


predicted: 1  truth: 1


predicted: 1  truth: 2


predicted: 1  truth: 6


predicted: 2  truth: 0


predicted: 2  truth: 2


predicted: 2  truth: 6


predicted: 1  truth: 0


predicted: 1  truth: 1


predicted: 1  truth: 6


predicted: 1  truth: 0


predicted: 1  truth: 1


predicted: 1  truth: 2


predicted: 2  truth: 1


predicted: 2  truth: 2


predicted: 2  truth: 7


predicted: 2  truth: 4


predicted: 2  truth: 2


predicted: 2  truth: 7


predicted: 1  truth: 4


predicted: 1  truth: 1


predicted: 1  truth: 7


predicted: 2  truth: 4


predicted: 4  truth: 1


predicted: 1  truth: 2


predicted: 9  truth: 1


predicted: 2  truth: 9


predicted: 9  truth: 10


predicted: 2  truth: 8


predicted: 2  truth: 9


predicted: 10  truth: 10


predicted: 8  truth:



predicted: 1  truth: 2


predicted: 1  truth: 6


predicted: 2  truth: 0


predicted: 2  truth: 2


predicted: 2  truth: 6


predicted: 1  truth: 0


predicted: 1  truth: 1


predicted: 1  truth: 6


predicted: 1  truth: 0


predicted: 0  truth: 1


predicted: 1  truth: 2


predicted: 2  truth: 1


predicted: 2  truth: 2


predicted: 2  truth: 7


predicted: 2  truth: 4


predicted: 2  truth: 2


predicted: 2  truth: 7


predicted: 1  truth: 4


predicted: 1  truth: 1


predicted: 1  truth: 7


predicted: 2  truth: 4


predicted: 4  truth: 1


predicted: 1  truth: 2


predicted: 10  truth: 1


predicted: 1  truth: 9


predicted: 9  truth: 10


predicted: 2  truth: 8


predicted: 2  truth: 9


predicted: 2  truth: 10


predicted: 8  truth: 8


predicted: 8  truth: 1


predicted: 8  truth: 10


predicted: 1  truth: 8


predicted: 9  truth: 1


predicted: 1  truth: 9


predicted: 10  truth: 1


predicted: 1  truth: 12


predicted: 1  truth: 10


predicted: 10  truth: 11


predicted: 10 

predicted: 1  truth: 5


predicted: 2  truth: 4


predicted: 4  truth: 1


predicted: 1  truth: 2


predicted: 1  truth: 1


predicted: 1  truth: 2


predicted: 1  truth: 6


predicted: 2  truth: 0


predicted: 2  truth: 2


predicted: 2  truth: 6


predicted: 1  truth: 0


predicted: 1  truth: 1


predicted: 1  truth: 6


predicted: 1  truth: 0


predicted: 0  truth: 1


predicted: 1  truth: 2


predicted: 2  truth: 1


predicted: 2  truth: 2


predicted: 2  truth: 7


predicted: 2  truth: 4


predicted: 2  truth: 2


predicted: 2  truth: 7


predicted: 1  truth: 4


predicted: 1  truth: 1


predicted: 1  truth: 7


predicted: 2  truth: 4


predicted: 4  truth: 1


predicted: 1  truth: 2


predicted: 10  truth: 1


predicted: 1  truth: 9


predicted: 1  truth: 10


predicted: 2  truth: 8


predicted: 2  truth: 9


predicted: 2  truth: 10


predicted: 10  truth: 8


predicted: 8  truth: 1


predicted: 10  truth: 10


predicted: 1  truth: 8


predicted: 1  truth: 1


predicted: 1  truth

...

predicted: 2  truth: 4


predicted: 4  truth: 1


predicted: 1  truth: 2


predicted: 2  truth: 1


predicted: 1  truth: 2


predicted: 2  truth: 6


predicted: 2  truth: 0


predicted: 2  truth: 2


predicted: 2  truth: 6


predicted: 1  truth: 0


predicted: 1  truth: 1


predicted: 1  truth: 6


predicted: 2  truth: 0


predicted: 0  truth: 1


predicted: 1  truth: 2


predicted: 2  truth: 1


predicted: 1  truth: 2


predicted: 2  truth: 7


predicted: 2  truth: 4


predicted: 2  truth: 2


predicted: 2  truth: 7


predicted: 1  truth: 4


predicted: 4  truth: 1


predicted: 1  truth: 7


predicted: 2  truth: 4


predicted: 4  truth: 1


predicted: 1  truth: 2


predicted: 10  truth: 1


predicted: 1  truth: 9


predicted: 9  truth: 10


predicted: 2  truth: 8


predicted: 2  truth: 9


predicted: 2  truth: 10


predicted: 10  truth: 8


predicted: 10  truth: 1


predicted: 1  truth: 10


predicted: 1  truth: 8


predicted: 1  truth: 1


predicted: 1  truth: 9


predicted: 10  trut

predicted: 1  truth: 4


predicted: 4  truth: 1


predicted: 1  truth: 5


predicted: 2  truth: 4


predicted: 4  truth: 1


predicted: 1  truth: 2


predicted: 2  truth: 1


predicted: 1  truth: 2


predicted: 2  truth: 6


predicted: 2  truth: 0


predicted: 2  truth: 2


predicted: 2  truth: 6


predicted: 1  truth: 0


predicted: 1  truth: 1


predicted: 1  truth: 6


predicted: 2  truth: 0


predicted: 0  truth: 1


predicted: 1  truth: 2


predicted: 2  truth: 1


predicted: 1  truth: 2


predicted: 2  truth: 7


predicted: 2  truth: 4


predicted: 2  truth: 2


predicted: 2  truth: 7


predicted: 1  truth: 4


predicted: 4  truth: 1


predicted: 1  truth: 7


predicted: 2  truth: 4


predicted: 4  truth: 1


predicted: 1  truth: 2


predicted: 10  truth: 1


predicted: 1  truth: 9


predicted: 9  truth: 10


predicted: 2  truth: 8


predicted: 2  truth: 9


predicted: 2  truth: 10


predicted: 10  truth: 8


predicted: 10  truth: 1


predicted: 1  truth: 10


predicted: 1  truth



predicted: 14  truth: 10


predicted: 10  truth: 13


predicted: 10  truth: 14


predicted: 10  truth: 10


predicted: 10  truth: 13


predicted: 13  truth: 1


predicted: 1  truth: 10


predicted: 1  truth: 13


predicted: 1  truth: 1


predicted: 1  truth: 14


[491/500] loss:2.100
predicted: 2  truth: 1


predicted: 1  truth: 2


predicted: 2  truth: 3


predicted: 10  truth: 0


predicted: 10  truth: 2


predicted: 10  truth: 3


predicted: 1  truth: 0


predicted: 1  truth: 1


predicted: 1  truth: 3


predicted: 2  truth: 0


predicted: 0  truth: 1


predicted: 1  truth: 2


predicted: 2  truth: 1


predicted: 1  truth: 2


predicted: 2  truth: 5


predicted: 10  truth: 4


predicted: 2  truth: 2


predicted: 2  truth: 5


predicted: 1  truth: 4


predicted: 4  truth: 1


predicted: 1  truth: 5


predicted: 2  truth: 4


predicted: 4  truth: 1


predicted: 1  truth: 2


predicted: 2  truth: 1


predicted: 1  truth: 2


predicted: 2  truth: 6


predicted: 2  truth: 0


predicted

predicted: 1  truth: 3


predicted: 2  truth: 0


predicted: 0  truth: 1


predicted: 1  truth: 2


predicted: 2  truth: 1


predicted: 1  truth: 2


predicted: 2  truth: 5


predicted: 10  truth: 4


predicted: 2  truth: 2


predicted: 2  truth: 5


predicted: 1  truth: 4


predicted: 4  truth: 1


predicted: 1  truth: 5


predicted: 2  truth: 4


predicted: 4  truth: 1


predicted: 1  truth: 2


predicted: 2  truth: 1


predicted: 1  truth: 2


predicted: 2  truth: 6


predicted: 2  truth: 0


predicted: 2  truth: 2


predicted: 2  truth: 6


predicted: 1  truth: 0


predicted: 1  truth: 1


predicted: 1  truth: 6


predicted: 2  truth: 0


predicted: 0  truth: 1


predicted: 1  truth: 2


predicted: 2  truth: 1


predicted: 1  truth: 2


predicted: 2  truth: 7


predicted: 2  truth: 4


predicted: 2  truth: 2


predicted: 2  truth: 7


predicted: 1  truth: 4


predicted: 4  truth: 1


predicted: 1  truth: 7


predicted: 2  truth: 4


predicted: 4  truth: 1


predicted: 1  truth: 2





predicted: 1  truth: 2


predicted: 2  truth: 1


predicted: 1  truth: 2


predicted: 2  truth: 6


predicted: 2  truth: 0


predicted: 2  truth: 2


predicted: 2  truth: 6


predicted: 1  truth: 0


predicted: 1  truth: 1


predicted: 1  truth: 6


predicted: 2  truth: 0


predicted: 0  truth: 1


predicted: 1  truth: 2


predicted: 2  truth: 1


predicted: 1  truth: 2


predicted: 2  truth: 7


predicted: 2  truth: 4


predicted: 2  truth: 2


predicted: 2  truth: 7


predicted: 1  truth: 4


predicted: 4  truth: 1


predicted: 1  truth: 7


predicted: 2  truth: 4


predicted: 4  truth: 1


predicted: 1  truth: 2


predicted: 10  truth: 1


predicted: 1  truth: 9


predicted: 9  truth: 10


predicted: 2  truth: 8


predicted: 2  truth: 9


predicted: 2  truth: 10


predicted: 10  truth: 8


predicted: 10  truth: 1


predicted: 1  truth: 10


predicted: 1  truth: 8


predicted: 1  truth: 1


predicted: 1  truth: 9


predicted: 10  truth: 1


predicted: 1  truth: 12


predicted: 12  

In [23]:
print("After training:")

print("W1:", W1)
print("W2:", W2)

After training:
W1: tensor([[ 0.8120, -0.9599,  1.6928,  1.8175,  0.9308,  1.9639,  1.7934,
          1.9439,  0.2413, -0.1096,  0.1486,  0.6768, -0.5316,  0.2111,
         -0.3897],
        [-1.7711,  0.5675, -0.4276,  0.2763, -1.3894, -0.0644,  0.2834,
         -0.0540,  0.5984, -0.5635, -0.2836,  1.1557, -1.5937, -0.2712,
         -1.5243],
        [ 0.9877,  0.2397, -0.1312, -0.9399, -1.2153,  0.7896, -0.9226,
          0.8071, -1.4991, -1.2812, -0.4115,  0.3227, -0.8812,  1.4440,
          0.5558],
        [-0.7126, -0.5245,  0.5953, -1.3419, -0.6583, -1.1324, -1.3328,
         -1.1270, -1.8807, -1.6694, -0.0260, -1.9226, -1.7569, -1.8046,
         -1.6889],
        [-0.4760, -0.4259,  0.1291, -0.7152, -0.9749, -0.4403, -0.6975,
         -0.4250,  0.9398,  0.7020,  1.8079,  1.2245,  0.6563,  1.2030,
          1.0441]])
W2: tensor([[ 1.6251,  1.0761, -1.1768, -0.0394, -0.4386],
        [ 1.9511, -0.9692, -0.7029, -1.6108,  1.2493],
        [ 1.4299, -0.4337, -0.1339, -1.6906, -1.10

In [38]:
input = idx_pairs[0][0]
target = idx_pairs[0][1]

print("Test input:", idx2word[input])

input = get_one_hot(input, word_vocab_size)
w1_out = torch.matmul(W1, input)
w2_out = torch.matmul(W2, w1_out)
log_softmax = F.log_softmax(w2_out, dim=0).view(1, -1)    

sorted = np.argsort(log_softmax.data[0]).numpy()

print("Test output:", end=' ')
for idx in sorted[::-1]:
    print(idx2word[idx], end=', ')

Test input: he
Test output: a, man, is, king, she, woman, queen, capital, paris, france, berlin, warsaw, he, germany, poland, 